# 📦 步骤 0：环境设置

这个 notebook 负责：
1. 安装所有必需的 Python 包
2. 设置项目目录结构（根据运行环境自动适配）
3. 配置文献文件路径
4. 配置 API 密钥和数据库连接

**⚠️ 重要**：
- 本 notebook 同时支持**真实 Colab** 和**本地 Jupyter/Colab 插件**
- 请按顺序执行所有单元格

## 1. 安装依赖包

In [1]:
%%capture
# 静默安装所有依赖（约需 2-3 分钟）
# 先卸载可能冲突的包
!pip uninstall -y numpy pandas

# 重新安装兼容版本（强制重新安装）
!pip install --upgrade --force-reinstall --no-cache-dir numpy==1.26.2
!pip install --upgrade --force-reinstall --no-cache-dir pandas==2.1.4

# 安装其他依赖
!pip install PyMuPDF==1.23.8
!pip install tqdm==4.66.1 pyyaml==6.0.1
!pip install spacy==3.7.2 jieba==0.42.1 scikit-learn==1.3.2 yake==0.4.8
!pip install keybert==0.8.5 sentence-transformers==2.3.1
!pip install neo4j==5.15.0
!pip install openai==1.6.1

# 下载 spaCy 中文模型
!python -m spacy download zh_core_web_sm

**⚠️ 重要提示**：如果上面的安装出现错误，请按以下步骤操作：

1. **重启运行时**：菜单栏 → 运行时 → 重启运行时
2. **重新运行本单元格**：从第一个单元格开始重新运行

这将清除所有缓存的包并重新安装干净的版本。

In [ ]:
# 验证安装
import sys
print(f"Python 版本: {sys.version}")

import fitz
import pandas as pd
import spacy
import openai
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer

print("\n✅ 所有依赖包安装成功！")

Python 版本: 3.10.13 (main, Apr 13 2025, 23:53:42) [Clang 17.0.0 (clang-1700.0.13.3)]


## 2. 检测运行环境

自动检测你是在真实的 Google Colab 还是本地 Jupyter 环境中运行

In [ ]:
import os
import sys

# 检测是否在 Colab 环境
try:
    import google.colab
    IS_COLAB = True
    ENV_TYPE = "Google Colab"
except ImportError:
    IS_COLAB = False
    ENV_TYPE = "本地 Jupyter"

print("=" * 60)
print(f"🌐 运行环境: {ENV_TYPE}")
print("=" * 60)

# 根据环境设置项目根目录
if IS_COLAB:
    # 真实 Colab 环境
    PROJECT_ROOT = '/content/PWD_Project'
    print(f"\n📁 项目根目录: {PROJECT_ROOT}")
    print("📝 文件上传方式: Google Drive / 直接上传 / 压缩包")
else:
    # 本地 Jupyter 环境
    # 使用当前 notebook 所在目录的父目录
    NOTEBOOK_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
    PROJECT_ROOT = os.path.dirname(NOTEBOOK_DIR) if 'notebooks' in NOTEBOOK_DIR else NOTEBOOK_DIR
    print(f"\n📁 项目根目录: {PROJECT_ROOT}")
    print("📝 文件上传方式: 本地文件系统（直接使用本地路径）")

print(f"\n✅ 环境检测完成")

🌐 运行环境: Google Colab

📁 项目根目录: /content/PWD_Project
📝 文件上传方式: Google Drive / 直接上传 / 压缩包

✅ 环境检测完成


In [ ]:
import os
from pathlib import Path

# 创建项目目录
os.makedirs(PROJECT_ROOT, exist_ok=True)

# 创建子目录
directories = [
    'pdf_files',      # PDF 文献（如果不使用本地路径）
    'output',         # 输出结果
    'cache',          # 缓存文件
    'config',         # 配置文件
    'modules'         # Python 模块
]

for directory in directories:
    dir_path = os.path.join(PROJECT_ROOT, directory)
    os.makedirs(dir_path, exist_ok=True)

print(f"✅ 项目目录已创建: {PROJECT_ROOT}")
print(f"\n目录结构:")
for directory in directories:
    print(f"  📁 {directory}/")

✅ 项目目录已创建: /content/PWD_Project

目录结构:
  📁 pdf_files/
  📁 output/
  📁 cache/
  📁 config/
  📁 modules/


## 4. 设置 PDF 文献路径

**📚 根据你的运行环境选择合适的方式**：

---

### 🏠 方式 D：本地文件系统（本地 Jupyter 环境推荐）

**适用场景**：在本地编译器使用 Colab/Jupyter 插件

**优点**：
- ✅ 直接使用本地文件路径，无需上传
- ✅ 适合大量文件
- ✅ 无需 Google Drive

**步骤**：直接运行下方的"方式 D"单元格，它会自动检测你本地的文献目录

---

### ☁️ 方式 A：Google Drive（真实 Colab 推荐）

**适用场景**：在 Google Colab 网页版运行

**步骤**：
1. 在 Google Drive 创建文件夹 `MyDrive/PWD`
2. 上传本地 `/Users/lifulin/Desktop/PWD/文献` 文件夹
3. 挂载 Drive 后运行"方式 A"单元格

---

### 📤 方式 B：直接上传（少量文件）

**适用场景**：文件数量少于 10 个

---

### 📦 方式 C：压缩包上传（中等数量）

**适用场景**：10-50 个文件

---

**👇 请根据你的环境运行对应的单元格**

In [ ]:
# ========== 方式 D：本地文件系统（本地 Jupyter 推荐）==========

import os

if not IS_COLAB:
    print("🏠 本地文件系统模式")
    print("=" * 60)
    
    # 自动检测本地文献路径（优先级排序）
    LOCAL_PDF_PATHS = [
        '/Users/lifulin/Desktop/PWD/文献',      # 你的实际路径（最高优先级）
        os.path.join(os.path.dirname(PROJECT_ROOT), '文献'),  # 项目同级目录
        os.path.join(PROJECT_ROOT, '文献'),                    # 项目内
        os.path.join(PROJECT_ROOT, 'pdf_files'),
        os.path.join(PROJECT_ROOT, 'PDFs'),
    ]
    
    PDF_DIR = None
    print("🔍 正在搜索本地 PDF 文献...")
    
    for pdf_path in LOCAL_PDF_PATHS:
        if os.path.exists(pdf_path):
            try:
                pdf_files = [f for f in os.listdir(pdf_path) if f.lower().endswith('.pdf')]
                if pdf_files:
                    PDF_DIR = pdf_path
                    print(f"\n✅ 自动找到 PDF 文献目录: {PDF_DIR}")
                    print(f"   包含 {len(pdf_files)} 个 PDF 文件:")
                    
                    # 显示文件列表和大小
                    total_size = 0
                    for i, pdf in enumerate(sorted(pdf_files)[:8], 1):
                        size_bytes = os.path.getsize(os.path.join(pdf_path, pdf))
                        size_kb = size_bytes / 1024
                        total_size += size_bytes
                        print(f"   {i}. {pdf} ({size_kb:.1f} KB)")
                    
                    if len(pdf_files) > 8:
                        remaining_files = pdf_files[8:]
                        remaining_size = sum(os.path.getsize(os.path.join(pdf_path, f)) for f in remaining_files)
                        total_size += remaining_size
                        print(f"   ... 还有 {len(pdf_files) - 8} 个文件")
                    
                    print(f"\n📊 总计: {len(pdf_files)} 个文件，{total_size / 1024 / 1024:.2f} MB")
                    break
            except PermissionError:
                print(f"   ⚠️ 无权限访问: {pdf_path}")
                continue
    
    if not PDF_DIR:
        print("\n❌ 未找到本地 PDF 文献")
        print("\n已尝试的路径:")
        for i, path in enumerate(LOCAL_PDF_PATHS, 1):
            status = "✅ 存在" if os.path.exists(path) else "❌ 不存在"
            print(f"   {i}. {path} [{status}]")
        
        # 手动输入模式
        print("\n" + "="*60)
        print("💡 请选择以下操作之一:")
        print("="*60)
        print("\n【选项 1】手动输入路径")
        
        manual_path = input("请输入 PDF 文献目录的完整路径（或按 Enter 跳过）: ").strip()
        
        if manual_path:
            # 处理可能的引号
            manual_path = manual_path.strip('\'"')
            
            if os.path.exists(manual_path):
                pdf_count = len([f for f in os.listdir(manual_path) if f.lower().endswith('.pdf')])
                if pdf_count > 0:
                    PDF_DIR = manual_path
                    print(f"\n✅ 已设置 PDF 目录: {PDF_DIR}")
                    print(f"   包含 {pdf_count} 个 PDF 文件")
                else:
                    print(f"⚠️ 路径存在但没有 PDF 文件: {manual_path}")
            else:
                print(f"❌ 路径不存在: {manual_path}")
        
        # 如果还是没有设置，使用默认目录
        if not PDF_DIR:
            print("\n【选项 2】使用默认目录")
            PDF_DIR = os.path.join(PROJECT_ROOT, 'pdf_files')
            os.makedirs(PDF_DIR, exist_ok=True)
            print(f"✅ 使用默认目录: {PDF_DIR}")
            print(f"   请将 PDF 文件复制到此目录")
    
    # 同步项目源代码（如果存在）
    PROJECT_SOURCE = '/Users/lifulin/Desktop/PWD'
    if os.path.exists(PROJECT_SOURCE) and PROJECT_SOURCE != PROJECT_ROOT:
        print(f"\n📋 检测到项目源码目录: {PROJECT_SOURCE}")
        
        try:
            # 同步 Python 模块
            source_modules = [f for f in os.listdir(PROJECT_SOURCE) 
                            if f.endswith('.py') and not f.startswith('__')]
            
            if source_modules:
                import shutil
                modules_dir = os.path.join(PROJECT_ROOT, 'modules')
                os.makedirs(modules_dir, exist_ok=True)
                
                synced = 0
                for py_file in source_modules:
                    src = os.path.join(PROJECT_SOURCE, py_file)
                    dst = os.path.join(modules_dir, py_file)
                    
                    # 只复制新文件或更新的文件
                    if not os.path.exists(dst) or os.path.getmtime(src) > os.path.getmtime(dst):
                        shutil.copy2(src, dst)
                        synced += 1
                
                if synced > 0:
                    print(f"   ✅ 已同步 {synced}/{len(source_modules)} 个 Python 模块")
                else:
                    print(f"   ℹ️  Python 模块已是最新（{len(source_modules)} 个）")
            
            # 同步配置文件
            config_src = os.path.join(PROJECT_SOURCE, 'config')
            config_dst = os.path.join(PROJECT_ROOT, 'config')
            
            if os.path.exists(config_src):
                os.makedirs(config_dst, exist_ok=True)
                config_files = [f for f in os.listdir(config_src) 
                              if os.path.isfile(os.path.join(config_src, f))]
                
                if config_files:
                    import shutil
                    for item in config_files:
                        src_file = os.path.join(config_src, item)
                        dst_file = os.path.join(config_dst, item)
                        shutil.copy2(src_file, dst_file)
                    print(f"   ✅ 已同步 {len(config_files)} 个配置文件")
                    
        except Exception as e:
            print(f"   ⚠️ 同步文件时出错: {e}")
    
    print("\n" + "=" * 60)
    print(f"✅ 本地文件系统配置完成")
    print(f"📁 PDF 目录: {PDF_DIR}")
    print("=" * 60)
    
else:
    print("⏭️  你在 Google Colab 环境，请使用方式 A、B 或 C")
    print("   跳过本单元格，运行下方对应的单元格")

⏭️  你在 Google Colab 环境，请使用方式 A、B 或 C
   跳过本单元格，运行下方对应的单元格


In [ ]:
# ========== 方式 A：从 Google Drive 复制（真实 Colab 推荐）==========

if IS_COLAB:
    print("☁️ Google Drive 模式")
    print("=" * 60)
    
    # 先挂载 Google Drive
    try:
        from google.colab import drive
        
        # 检查是否已挂载
        if not os.path.exists('/content/drive/MyDrive'):
            print("正在挂载 Google Drive...")
            drive.mount('/content/drive')
        else:
            print("✅ Google Drive 已挂载")
        
        # 查找 PDF 目录
        DRIVE_PROJECT_PATH = '/content/drive/MyDrive/PWD'
        
        PDF_DIR_OPTIONS = [
            f"{DRIVE_PROJECT_PATH}/文献",
        ]
        
        PDF_DIR = None
        for pdf_path in PDF_DIR_OPTIONS:
            if os.path.exists(pdf_path):
                pdf_files = [f for f in os.listdir(pdf_path) if f.endswith('.pdf')]
                if pdf_files:
                    PDF_DIR = pdf_path
                    print(f"\n✅ 找到 PDF 文献目录: {PDF_DIR}")
                    print(f"   包含 {len(pdf_files)} 个 PDF 文件:")
                    for i, pdf in enumerate(pdf_files[:5], 1):
                        print(f"   {i}. {pdf}")
                    if len(pdf_files) > 5:
                        print(f"   ... 还有 {len(pdf_files) - 5} 个文件")
                    break
        
        if not PDF_DIR:
            print(f"\n⚠️  未在 Drive 中找到 PDF 文献")
            print("\n请按以下步骤操作:")
            print("1. 访问 drive.google.com")
            print("2. 创建文件夹 'PWD/文献'")
            print(f"3. 上传 PDF 文件到该文件夹")
            print("4. 重新运行本单元格")
            
            PDF_DIR = f"{DRIVE_PROJECT_PATH}/文献"
            print(f"\n将使用默认路径: {PDF_DIR}")
        
        # 复制项目文件
        if os.path.exists(DRIVE_PROJECT_PATH):
            py_files = [f for f in os.listdir(DRIVE_PROJECT_PATH) if f.endswith('.py')]
            if py_files:
                for py_file in py_files:
                    src = os.path.join(DRIVE_PROJECT_PATH, py_file)
                    dst = os.path.join(PROJECT_ROOT, 'modules', py_file)
                    import shutil
                    shutil.copy2(src, dst)
                print(f"✅ 复制了 {len(py_files)} 个 Python 文件")
            
            config_path = os.path.join(DRIVE_PROJECT_PATH, 'config')
            if os.path.exists(config_path):
                for item in os.listdir(config_path):
                    if os.path.isfile(os.path.join(config_path, item)):
                        src = os.path.join(config_path, item)
                        dst = os.path.join(PROJECT_ROOT, 'config', item)
                        import shutil
                        shutil.copy2(src, dst)
                print(f"✅ 复制了配置文件")
        
        print("\n" + "=" * 60)
        print("✅ Google Drive 配置完成")
        print("=" * 60)
        
    except Exception as e:
        print(f"❌ Google Drive 挂载失败: {e}")
        print("\n请尝试:")
        print("1. 刷新页面重试")
        print("2. 或使用方式 B/C 直接上传文件")
        print("3. 详细故障排除：查看 notebooks/故障排除指南.md")
        
        PDF_DIR = f"{PROJECT_ROOT}/pdf_files"
        print(f"\n临时使用: {PDF_DIR}")

else:
    print("⏭️  你在本地 Jupyter 环境，请使用方式 D")
    print("   跳过本单元格，运行上方的'方式 D'单元格")

☁️ Google Drive 模式
正在挂载 Google Drive...
❌ Google Drive 挂载失败: mount failed

请尝试:
1. 刷新页面重试
2. 或使用方式 B/C 直接上传文件
3. 详细故障排除：查看 notebooks/故障排除指南.md

临时使用: /content/PWD_Project/pdf_files


In [ ]:
# ========== 方式 B：直接上传 PDF 文件 ==========

if IS_COLAB:
    from google.colab import files
    
    print("🔼 请选择要上传的 PDF 文件（可多选）")
    print("提示：按 Ctrl 或 Cmd 可以选择多个文件\n")
    
    uploaded = files.upload()
    
    if uploaded:
        # 移动到 PDF 目录
        PDF_DIR = f"{PROJECT_ROOT}/pdf_files"
        os.makedirs(PDF_DIR, exist_ok=True)
        
        pdf_count = 0
        for filename, content in uploaded.items():
            if filename.endswith('.pdf'):
                filepath = os.path.join(PDF_DIR, filename)
                with open(filepath, 'wb') as f:
                    f.write(content)
                pdf_count += 1
                print(f"✅ 已上传: {filename} ({len(content)/1024:.1f} KB)")
        
        print(f"\n✅ 总共上传了 {pdf_count} 个 PDF 文件到: {PDF_DIR}")
    else:
        print("⚠️  未选择文件")
else:
    print("⏭️  本单元格仅适用于 Google Colab 环境")
    print("   你在本地环境，请使用方式 D")

🔼 请选择要上传的 PDF 文件（可多选）
提示：按 Ctrl 或 Cmd 可以选择多个文件



KeyboardInterrupt: 

In [ ]:
# ========== 方式 C：上传压缩包 ==========

if IS_COLAB:
    from google.colab import files
    import zipfile
    
    print("🔼 请上传压缩包:")
    print("1. 文献压缩包（文献.zip 或 pdfs.zip）")
    print("2. 项目代码压缩包（可选）\n")
    
    uploaded = files.upload()
    
    if uploaded:
        for filename in uploaded.keys():
            print(f"\n📦 正在解压: {filename}")
            
            try:
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    # 获取压缩包内容
                    file_list = zip_ref.namelist()
                    pdf_files = [f for f in file_list if f.endswith('.pdf')]
                    py_files = [f for f in file_list if f.endswith('.py')]
                    
                    if pdf_files:
                        # 解压 PDF 文件
                        PDF_DIR = f"{PROJECT_ROOT}/pdf_files"
                        os.makedirs(PDF_DIR, exist_ok=True)
                        
                        for pdf_file in pdf_files:
                            # 只提取文件名，不保留目录结构
                            pdf_name = os.path.basename(pdf_file)
                            if pdf_name:  # 跳过目录项
                                zip_ref.extract(pdf_file, PDF_DIR)
                                # 如果在子目录中，移动到根目录
                                source = os.path.join(PDF_DIR, pdf_file)
                                target = os.path.join(PDF_DIR, pdf_name)
                                if source != target and os.path.exists(source):
                                    import shutil
                                    shutil.move(source, target)
                        
                        print(f"   ✅ 解压了 {len(pdf_files)} 个 PDF 文件到: {PDF_DIR}")
                    
                    if py_files:
                        # 解压 Python 文件
                        zip_ref.extractall(f'{PROJECT_ROOT}/modules')
                        print(f"   ✅ 解压了 {len(py_files)} 个 Python 文件")
                    
                    # 检查是否有 config 文件夹
                    config_files = [f for f in file_list if 'config/' in f or 'config\\' in f]
                    if config_files:
                        for config_file in config_files:
                            zip_ref.extract(config_file, PROJECT_ROOT)
                        print(f"   ✅ 解压了配置文件")
                    
                # 删除压缩包
                os.remove(filename)
                print(f"   ✅ {filename} 处理完成")
                
            except Exception as e:
                print(f"   ❌ 解压失败: {e}")
        
        # 确认最终的 PDF 目录
        if 'PDF_DIR' not in locals():
            PDF_DIR = f"{PROJECT_ROOT}/pdf_files"
        
        if os.path.exists(PDF_DIR):
            final_pdfs = [f for f in os.listdir(PDF_DIR) if f.endswith('.pdf')]
            print(f"\n✅ PDF 目录已设置: {PDF_DIR}")
            print(f"   包含 {len(final_pdfs)} 个 PDF 文件")
    else:
        print("⚠️  未选择文件")
else:
    print("⏭️  本单元格仅适用于 Google Colab 环境")
    print("   你在本地环境，请使用方式 D")

## 4.5 验证 PDF 文献加载

运行下面的单元格确认 PDF 文件已正确加载

In [ ]:
# 验证 PDF 文献
print("=" * 60)
print("📚 PDF 文献验证")
print("=" * 60)

if 'PDF_DIR' not in locals() or not PDF_DIR:
    print("\n❌ PDF_DIR 未设置")
    print("\n请根据你的环境选择:")
    if IS_COLAB:
        print("  - 方式 A：Google Drive（大量文件）")
        print("  - 方式 B：直接上传（少量文件）")
        print("  - 方式 C：压缩包上传（中等数量）")
    else:
        print("  - 方式 D：本地文件系统（推荐）")
    print("\n运行对应的单元格后再执行本验证")
else:
    print(f"\n📁 PDF 目录: {PDF_DIR}")
    print(f"🌐 运行环境: {ENV_TYPE}")
    
    if not os.path.exists(PDF_DIR):
        print(f"\n❌ 目录不存在")
        print(f"\n建议操作：")
        if IS_COLAB:
            print(f"1. 检查 Google Drive 路径是否正确")
            print(f"2. 或使用方式 B/C 直接上传 PDF 文件")
        else:
            print(f"1. 检查本地路径是否正确")
            print(f"2. 或在方式 D 中手动输入正确路径")
    else:
        pdf_files = [f for f in os.listdir(PDF_DIR) if f.endswith('.pdf')]
        
        if not pdf_files:
            print(f"\n⚠️  目录存在但没有 PDF 文件")
            print(f"\n请上传 PDF 文件到此目录")
        else:
            print(f"\n✅ 找到 {len(pdf_files)} 个 PDF 文件\n")
            
            # 显示文件列表
            print("文件列表:")
            for i, pdf in enumerate(pdf_files[:10], 1):  # 显示前10个
                file_path = os.path.join(PDF_DIR, pdf)
                file_size = os.path.getsize(file_path) / 1024  # KB
                print(f"  {i}. {pdf} ({file_size:.1f} KB)")
            
            if len(pdf_files) > 10:
                print(f"  ... 还有 {len(pdf_files) - 10} 个文件")
            
            # 计算总大小
            total_size = sum(os.path.getsize(os.path.join(PDF_DIR, f)) for f in pdf_files) / 1024 / 1024  # MB
            print(f"\n总大小: {total_size:.2f} MB")
            
            print(f"\n✅ PDF 文献已准备就绪，可以继续下一步")

## 5. 配置参数

**⚠️ 重要**：请在运行前填写以下配置

In [ ]:
import yaml

# 确保 PDF_DIR 已设置
if 'PDF_DIR' not in locals() or not PDF_DIR:
    print("⚠️ 警告: PDF_DIR 未设置，使用默认值")
    PDF_DIR = os.path.join(PROJECT_ROOT, 'pdf_files')

# Colab/Jupyter 通用配置
CONFIG = {
    # PDF 配置
    'pdf': {
        'input_directory': PDF_DIR,  # 使用上面设置的 PDF_DIR
        'enable_cache': True
    },
    
    # LLM 配置（使用 OpenAI）
    'llm': {
        'provider': 'openai',
        'model': 'gpt-3.5-turbo',  # 或 'gpt-4' (更贵但效果更好)
        'api_key': '',  # ⚠️⚠️⚠️ 在此填入你的 OpenAI API Key
        'max_chunks': 30,  # 限制处理块数以控制成本（30块约 $1-2）
        'temperature': 0.1,
        'timeout': 120
    },
    
    # 去重配置
    'deduplication': {
        'similarity_threshold': 0.85,
        'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2'
    },
    
    # 过滤配置
    'filtering': {
        'min_importance': 1,
        'min_connections': 0
    },
    
    # Neo4j 配置（使用云端实例）
    'neo4j': {
        'uri': '',  # ⚠️⚠️⚠️ Neo4j Aura URI，如 'neo4j+s://xxxxx.databases.neo4j.io'
        'user': 'neo4j',
        'password': '',  # ⚠️⚠️⚠️ Neo4j 密码
        'database': 'neo4j'
    },
    
    # 输出配置
    'output': {
        'base_directory': os.path.join(PROJECT_ROOT, 'output'),
        'cache_directory': os.path.join(PROJECT_ROOT, 'cache')
    },
    
    # 系统配置
    'system': {
        'enable_cache': True,
        'enable_parallel': False,  # 单核环境不建议并行
        'environment': ENV_TYPE  # 记录运行环境
    }
}

# 保存配置到文件
config_path = os.path.join(PROJECT_ROOT, 'config', 'config_colab.yaml')
os.makedirs(os.path.dirname(config_path), exist_ok=True)

with open(config_path, 'w', encoding='utf-8') as f:
    yaml.dump(CONFIG, f, allow_unicode=True)

print("✅ 配置已保存到:", config_path)

# 验证关键配置
print("\n📋 配置检查:")
print(f"🌐 运行环境: {ENV_TYPE}")

if not CONFIG['llm']['api_key']:
    print("❌ OpenAI API Key 未设置")
    print("   获取地址: https://platform.openai.com/api-keys")
else:
    print(f"✅ OpenAI API Key: {'*' * 20}{CONFIG['llm']['api_key'][-4:]}")

if not CONFIG['neo4j']['uri'] or not CONFIG['neo4j']['password']:
    print("⚠️  Neo4j 未配置（步骤 4 时需要）")
    print("   免费实例: https://neo4j.com/cloud/aura-free/")
else:
    print(f"✅ Neo4j URI: {CONFIG['neo4j']['uri']}")

print(f"\n📁 PDF 目录: {CONFIG['pdf']['input_directory']}")
print(f"📁 输出目录: {CONFIG['output']['base_directory']}")

## 6. 保存环境变量（供其他 notebook 使用）

In [ ]:
# 将关键路径保存为环境变量
os.environ['PWD_PROJECT_ROOT'] = PROJECT_ROOT
os.environ['PWD_CONFIG_PATH'] = config_path
os.environ['PWD_PDF_DIR'] = CONFIG['pdf']['input_directory']
os.environ['PWD_OUTPUT_DIR'] = CONFIG['output']['base_directory']

# 保存到文件（Colab 会话间共享）
env_file = f'{PROJECT_ROOT}/.env'
with open(env_file, 'w') as f:
    f.write(f"PROJECT_ROOT={PROJECT_ROOT}\n")
    f.write(f"CONFIG_PATH={config_path}\n")
    f.write(f"PDF_DIR={CONFIG['pdf']['input_directory']}\n")
    f.write(f"OUTPUT_DIR={CONFIG['output']['base_directory']}\n")

print("✅ 环境变量已保存")
print(f"\n后续 notebook 将自动读取配置")

## 7. 环境设置完成

### ✅ 检查清单

- [x] 依赖包已安装
- [x] 运行环境已检测
- [x] 项目目录已创建
- [x] PDF 文献路径已设置
- [x] 配置文件已创建
- [x] 环境变量已保存

### 📊 当前配置摘要

```python
运行环境: {ENV_TYPE}
项目根目录: {PROJECT_ROOT}
PDF 目录: {PDF_DIR if 'PDF_DIR' in locals() else '未设置'}
配置文件: {config_path if 'config_path' in locals() else '未生成'}
```

### 📝 下一步

请打开 **`01_PDF提取.ipynb`** 继续执行 PDF 文本提取

### 💡 提示

**本地 Jupyter 环境**：
- ✅ 可以直接使用本地文件路径，无需上传
- ✅ 修改代码后重新运行即可生效
- ✅ 所有输出文件保存在项目根目录

**Google Colab 环境**：
- ⚠️ 会话超时后需要重新运行本 notebook
- ⚠️ 建议将重要文件保存到 Google Drive
- ⚠️ 注意 API 使用费用

**修改配置**：
- 如需修改 API Key 或其他配置，重新运行「步骤 5」即可
- PDF 文件可以随时添加到 `PDF_DIR` 目录